# Take 3 just combine context with input and something to indicate speration like end start token in between

# Take 4 remove the concept of feeding the output all together just give input in encoder and context in decoder (all outputs/preds come at once)

# Take 5 diffrent embedding layer for context

# Take 6 pos tagging and ner 

# Imports and setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow_datasets
!pip3 install tensorflow-gpu --upgrade --force-reinstall
!pip install tensorflow_text
!pip3 install --quiet tensorflow-text

import logging 
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_text 
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import common_functions as cf
import tensorflow_hub as hub
import os
import tensorflow_hub as hub
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger('tensorflow').setLevel(logging.ERROR)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow_gpu-2.9.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (511.7 MB)
  Using cached tensorflow_io_gcs_filesystem-0.26.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.4 MB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached setuptools-62.6.0-py3-none-any.whl (1.2 MB)
  Using cached h5py-3.7.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (4.1 MB)
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached libclang-14.0.1-py2.py3-none-manylinux1_x86_64.whl (14.5 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached protobuf-3.19.4-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached astunp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Misc Area


In [ ]:
def modifyLine(line):
  line = line.replace("\'ll", " will")
  line = line.replace("\'re", " are")
  line = line.replace("\'m", " am")
  line = line.replace("\'d", " would")
  line = line.replace("n\'t", " not")
  line = line.replace("\'ve", " have")
  line = line.replace("\'s", " \'s")
  line = line.replace("s\'", " \'s")

  return line

#convert multiple continous dots into three continous dots

file = open('/content/well.txt', 'r')
allLines = file.readlines()
string = ""
for line in allLines:  
  string += line
string = string.replace("\n", "")
prepared_lines = []
temp_string = string[0]
for i in range(1,len(string)):
  if i< len(string)-2:
    if (string[i] =='.' or string[i]==":") and ( (string[i+1].isspace() and (not string[i+2].islower())) or (string[i-1].islower() and (string[i+1].isupper() or string[i+1]=="\""))):
      temp_string += string[i]
      prepared_lines.append(modifyLine(temp_string))
      temp_string = ""
    else:
      temp_string += string[i]   
    if string[i] =='.' and string[i+1] =='.' and string[i+2] =='.':  
      temp_string += string[i]
      prepared_lines.append(modifyLine(temp_string))
      temp_string = ""
      i=i+2
  else:
    if string[i] =='.':
      temp_string += string[i]
      prepared_lines.append(modifyLine(temp_string))
      temp_string = ""
    else:
      temp_string += string[i]

verb_forms_df = pd.read_csv("verbs.txt")

a = verb_forms_df["PresentTense"].str.lower()
b = verb_forms_df["PastTense"].str.lower()
c = verb_forms_df["PastParticiple"].str.lower()
d = verb_forms_df["PresentParticiple"].str.lower()

def ruin_grammer(line):
  for i, word in enumerate(b):
    line = line.replace(" "+word+" "," "+a[i]+" ")
  for i, word in enumerate(c):
    line = line.replace(" "+word+" "," "+a[i]+" ")
  for i, word in enumerate(d):
    line = line.replace(" "+word+" "," "+a[i]+" ")
  line = line.replace(" a ", " ")
  line = line.replace(" an ", " ")
  line = line.replace(" from ", " ")
  line = line.replace(" the ", " ")
  line = line.replace(" on ", " ")
  line = line.replace(" in ", " ")
  line = line.replace(" at ", " ")
  line = line.replace(" of ", " ")
  line = line.replace(" is ", " ")
  line = line.replace(" am ", " ")
  line = line.replace(" to ", " ")
  line = line.replace(" was ", " ")
  line = line.replace(" were ", " ")
  line = line.replace(" it ", " ")
  line = line.replace(" your ", " you ")
  line = line.replace(" and ", " ")
  line = line.replace(" or ", " ")
      
  return line

In [ ]:
# assuming prepared lines are modified and standardized 

max_words = 128
lengths = []

for i in prepared_lines:
  lengths.append(len(i.split()))

train_context_misc=[] 
train_input_misc=[] 
train_target_misc = []

context = ""
for i, line in enumerate(prepared_lines):
  temp_context = ""
  train_context_misc.append(context)
  train_input_misc.append(context+"[END] [START]"+ruin_grammer(line))
  overflow = 2 + len(context.split()) + len(line.split()) - max_words
  if overflow<0:
    overflow = 0
  splits = context.split()
  for index in range(overflow, len(splits)):
    temp_context += splits[index]+" "
  context = temp_context + line
  train_target_misc.append(line)

# Text tokenization and detokenization

In [ ]:
model_name = 'ted_hrlr_translate_pt_en_converter'
tf.keras.utils.get_file(
    f'{model_name}.zip',
    f'https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip',
    cache_dir='.', cache_subdir='', extract=True
)

tokenizer = tf.saved_model.load(model_name).en
MAX_TOKENS = 256

# Getting data and prepraing Input Pipeline

In [ ]:
f_c= open(r'/content/drive/MyDrive/train_context', 'r')
train_context_book = f_c.readlines()

f_i= open(r'/content/drive/MyDrive/train_input', 'r')
train_input_book = f_i.readlines()

f_t= open(r'/content/drive/MyDrive/train_target', 'r')
train_target_book = f_t.readlines()

for i in range(len(train_context_book)):
  train_context_book[i] = train_context_book[i].replace("\n","")
  train_input_book[i] = train_input_book[i].replace("\n","")
  train_target_book[i] = train_target_book[i].replace("\n","")

f_c.close()
f_i.close()
f_t.close()

In [ ]:
#cf.unzip_data("/content/drive/MyDrive/c415.zip")
#X_c4 = []
#y_c4 = []
#x = True
#for i in range(25):
#  file1 = open(f'/content/{i}.csv', 'r')
#  for line in file1.readlines():
#    if line.count("value:") >0:
#      line = line.replace(": \"", ": ")
#      line = line.replace(".\"\n", ".")
#      line = line.replace("?\"\n", "?")
#      line = line.replace("!\"\n", "!")
#      line = line.replace("\\'s"," is")
#      line = line.replace("\\'re"," are")
#
#      if x:
#        line = line.split("value: ")[1]
#        line = line.replace(" of "," ")
#        line = line.replace(" am "," ")
#        line = line.replace(" are "," ")
#        line = line.replace(" to "," ")
#        line = line.replace(" be "," ")
#        line = line.replace(" from ","")
#        line = modifyLine(line)
#        #line = ruin_grammer(line)
#
#
#        X_c4.append(line)
#
#        x = False
#      else:
#        y_c4.append(line.split("value: ")[1])
#        x = True 
#
#len(X_c4), len(y_c4)
#X_c4[-10:]
#y_c4[-10:]
#X_c4_selected = []
#y_c4_selected= []
#for i, line in enumerate(X_c4):
#  if not line.count("\\")>0:
#    X_c4_selected.append(line)
#    y_c4_selected.append(y_c4[i])

In [ ]:
#X_df1 = pd.read_csv("input1.csv")
#X_df2 = pd.read_csv("input2.csv")

#y_df1 = pd.read_csv("target1.csv")
#y_df2 = pd.read_csv("target2.csv")
#
#X_and_y_df3 = pd.read_csv("inputAndTarget3.txt", header=None)
#
#X1 = X_df1["sentence"].to_numpy()
#X2 = X_df2[" sentence"].to_numpy()
#X3 = X_and_y_df3[1].to_numpy()
#
#y1 = y_df1["sentence"].to_numpy()
#y2 = y_df2["sentence"].to_numpy()
#y3 = X_and_y_df3[1].to_numpy()
#
#split_size1 = int( len(X1) * .85 )
#split_size2 = int( len(X2) * .85 )
#split_size3 = int( len(X3) * .80 )
#
#X1_train, X1_val = X1[:split_size1], X1[split_size1:]
#X2_train, X2_val = X2[:split_size2], X2[split_size2:]
#X3_train, X3_val = X3[:split_size3], X3[split_size3:]
#
#y1_train, y1_val = y1[:split_size1], y1[split_size1:]
#y2_train, y2_val = y2[:split_size2], y2[split_size2:]
#y3_train, y3_val = y3[:split_size3], y3[split_size3:]
#
#train_sentences = np.concatenate((X1, X3)) 
#train_targets = np.concatenate((y1, y3))
#train_contexts = []
#
#for line in train_sentences:
##  train_contexts.append("")
#
#
#train_context_book = np.concatenate((train_context_book[500000:1000000], train_contexts)) 
#train_input_book = np.concatenate((train_input_book[500000:1000000], train_sentences)) 
#train_target_book = np.concatenate((train_target_book[500000:1000000], train_targets)) 
#val_sentences = np.concatenate((X1_val, X3_val))
#val_targets = np.concatenate((y1_val, y3_val))

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE= 64
MAX_TOKENS = 256

def filter_max_tokens(context, inp, tar):
  num_tokens = tf.maximum(tf.shape(tar)[1], tf.shape(inp)[1])
  num_tokens = tf.maximum(tf.shape(context)[1], num_tokens)

  return num_tokens < MAX_TOKENS

def tokenize_pairs(context, inp, tar): 
  context = tokenizer.tokenize(context)
  inp = tokenizer.tokenize(inp)
  tar = tokenizer.tokenize(tar)
  return context.to_tensor(), inp.to_tensor(), tar.to_tensor()

def make_batches(ds):
  return (
      ds
      .cache()
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(tokenize_pairs, num_parallel_calls=tf.data.AUTOTUNE)
      .filter(filter_max_tokens)
      .prefetch(tf.data.AUTOTUNE))

train_ds = tf.data.Dataset.from_tensor_slices((train_context_book, train_input_book, train_target_book))
#val_ds = tf.data.Dataset.from_tensor_slices((train_input_misc,train_context_misc, train_target_misc))

train_ds = make_batches(train_ds)
#val_ds = make_batches(val_ds)

# Position encoding and masking

In [ ]:
def get_angles(pos, i , d_model):
  angle_rate = 1/np.power(10000, 2*(i//2) / np.float32(d_model))
  return pos/angle_rate

def positional_encoding(pos, d_model):
  angle_rads = get_angles( np.arange(pos)[:, np.newaxis] , np.arange(d_model)[np.newaxis,:], d_model)
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
  pos_encdoing = angle_rads[np.newaxis, ...]
  return tf.cast(pos_encdoing, dtype=tf.float32)

In [ ]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  return seq[:, tf.newaxis, tf.newaxis, :]

def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size,size)), -1, 0)
  return mask

# MultiHead Attention

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

# Encoder and Decoder Layers

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model=d_model, num_heads=num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
    self.mha2 = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
    self.mhac = MultiHeadAttention(d_model=d_model, num_heads=num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernormc = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)
    self.dropoutc = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, context, training,
           look_ahead_mask, padding_mask, padding_mask_context):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)

    attn2, attn_weights_block2 = self.mha2(
        enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
    attn2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

    attnc, attn_weights_blockc = self.mhac(
        context, context, out2, padding_mask_context)  # (batch_size, target_seq_len, d_model)
    attnc = self.dropoutc(attnc, training=training)
    outc = self.layernorm2(attnc + out2)  # (batch_size, target_seq_len, d_model)

    ffn_output = self.ffn(outc)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output, training=training)
    out3 = self.layernorm3(ffn_output + outc)  # (batch_size, target_seq_len, d_model)

    return out3, attn_weights_block1, attn_weights_block2

# Encoder and Decoder

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self,*, num_layers, d_model, num_heads, dff, input_vocab_size,
               rate=0.1, embedding_layer):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(MAX_TOKENS, self.d_model)

    self.enc_layers = [
        EncoderLayer(d_model=d_model, num_heads=num_heads, dff=dff, rate=rate)
        for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]

    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    return x  # (batch_size, input_seq_len, d_model)

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self,*, num_layers, d_model, num_heads, dff, target_vocab_size,
               rate=0.1, embedding_layer):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(MAX_TOKENS, d_model)

    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads, dff=dff, rate=rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, context, training,
           look_ahead_mask, padding_mask, padding_mask_context):

    seq_len = tf.shape(x)[1]
    attention_weights = {}

    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    context = self.embedding(context)  # (batch_size, context_seq_length, d_model)
    context *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    context += self.pos_encoding[:, :tf.shape(context)[1], :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, context, training,
                                             look_ahead_mask, padding_mask, padding_mask_context)

      attention_weights[f'decoder_layer{i+1}_block1'] = block1
      attention_weights[f'decoder_layer{i+1}_block2'] = block2

    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

# Transformer

In [ ]:
class Transformer(tf.keras.Model):
  def __init__(self,*, num_layers, d_model, num_heads, dff, input_vocab_size,
               target_vocab_size, rate=0.1):
    super().__init__()
    self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           input_vocab_size=input_vocab_size, rate=rate, embedding_layer=EMBEDDING_LAYER)

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           target_vocab_size=target_vocab_size, rate=rate, embedding_layer=EMBEDDING_LAYER)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs, training):
    # Keras models prefer if you pass all your inputs in the first argument
    context, inp, tar = inputs

    padding_mask, look_ahead_mask, padding_mask_context = self.create_masks(context, inp, tar)

    enc_output = self.encoder(inp, training, padding_mask)  # (batch_size, inp_seq_len, d_model)

    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    dec_output, attention_weights = self.decoder(
        tar, enc_output, context, training, look_ahead_mask, padding_mask, padding_mask_context)

    final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

    return final_output, attention_weights

  def create_masks(self, context, inp, tar):
    # Encoder padding mask (Used in the 2nd attention block in the decoder too.)
    padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    look_ahead_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return padding_mask, look_ahead_mask, create_padding_mask(context)

# Hyperparameters and embedding layer

In [ ]:
num_layers = 4
d_model = 512
dff = 2048
num_heads = 8
dropout_rate = 0.1

EMBEDDING_LAYER = layers.Embedding(tokenizer.get_vocab_size().numpy() ,d_model)

# Optimizer with custom learing rate

In [ ]:
class CustomSchedule(keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps = 4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model 
    self.d_model = tf.cast(self.d_model, dtype= tf.float32)
    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = step*1.35
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1 = 0.9 ,beta_2 = 0.98, epsilon = 1e-9)

# Loss and Metrices

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


def accuracy_function(real, pred):
  accuracies = tf.equal(real, tf.argmax(pred, axis=2))

  mask = tf.math.logical_not(tf.math.equal(real, 0))
  accuracies = tf.math.logical_and(mask, accuracies)

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

# Training preprations

In [ ]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=tokenizer.get_vocab_size().numpy(),
    target_vocab_size=tokenizer.get_vocab_size().numpy(),
    rate=dropout_rate)

In [ ]:
checkpoint_path = 'drive/MyDrive/twct2wc_checkpoints/train'
ckpt = tf.train.Checkpoint(transformer = transformer, optimizer = optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep = 5)
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print('latest checkpoint restored')

latest checkpoint restored


In [ ]:
train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]


@tf.function(input_signature=train_step_signature)
def train_step(context, inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]

  with tf.GradientTape() as tape:
    predictions, _ = transformer([context, inp, tar_inp],
                                 training = True)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

  train_loss(loss)
  train_accuracy(accuracy_function(tar_real, predictions))

# Actual training

In [ ]:
EPOCHS = 1

for epoch in range(EPOCHS):
  start = time.time()

  train_loss.reset_states()
  train_accuracy.reset_states()

  for (batch, (context, inp, tar)) in enumerate(train_ds):
    train_step(context, inp, tar)

    if batch % 100 == 0:
      print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  if (epoch + 1) % 1 == 0:
    ckpt_save_path = ckpt_manager.save()
    print(f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')

  print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')

Epoch 1 Batch 0 Loss 0.2134 Accuracy 0.9288
Epoch 1 Batch 100 Loss 0.1946 Accuracy 0.9379
Epoch 1 Batch 200 Loss 0.1893 Accuracy 0.9393
Epoch 1 Batch 300 Loss 0.1896 Accuracy 0.9391
Epoch 1 Batch 400 Loss 0.1893 Accuracy 0.9390
Epoch 1 Batch 500 Loss 0.1904 Accuracy 0.9389
Epoch 1 Batch 600 Loss 0.1923 Accuracy 0.9385
Epoch 1 Batch 700 Loss 0.1920 Accuracy 0.9388
Epoch 1 Batch 800 Loss 0.1926 Accuracy 0.9387
Epoch 1 Batch 900 Loss 0.1920 Accuracy 0.9389
Epoch 1 Batch 1000 Loss 0.1910 Accuracy 0.9392
Epoch 1 Batch 1100 Loss 0.1902 Accuracy 0.9394
Epoch 1 Batch 1200 Loss 0.1896 Accuracy 0.9396
Epoch 1 Batch 1300 Loss 0.1906 Accuracy 0.9394
Epoch 1 Batch 1400 Loss 0.1931 Accuracy 0.9387
Epoch 1 Batch 1500 Loss 0.1937 Accuracy 0.9386
Epoch 1 Batch 1600 Loss 0.1949 Accuracy 0.9384
Epoch 1 Batch 1700 Loss 0.1975 Accuracy 0.9377
Epoch 1 Batch 1800 Loss 0.1990 Accuracy 0.9372
Epoch 1 Batch 1900 Loss 0.2004 Accuracy 0.9367
Epoch 1 Batch 2000 Loss 0.2013 Accuracy 0.9363
Epoch 1 Batch 2100 Loss 0

# Getting output

In [ ]:
class Translator(tf.Module):
  def __init__(self, tokenizer, transformer):
    self.tokenizer = tokenizer
    self.transformer = transformer

  def __call__(self, sentence, max_length=MAX_TOKENS):

    assert isinstance(sentence, tf.Tensor)
    if len(sentence.shape) == 0:
      sentence = sentence[tf.newaxis]

    sentence = self.tokenizer.tokenize(sentence).to_tensor()

    encoder_input = sentence

    start_end = self.tokenizer.tokenize([''])[0]
    start = start_end[0][tf.newaxis]
    end = start_end[1][tf.newaxis]

    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)

    for i in tf.range(max_length):
      output = tf.transpose(output_array.stack())
      predictions, _ = self.transformer([tokenizer.tokenize(["I am fine"]).to_tensor(), encoder_input, output], training=False)

      # select the last token from the seq_len dimension
      predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)

      predicted_id = tf.argmax(predictions, axis=-1)

      # concatentate the predicted_id to the output which is given to the decoder
      # as its input.
      output_array = output_array.write(i+1, predicted_id[0])

      if predicted_id == end:
        break

    output = tf.transpose(output_array.stack())
    # output.shape (1, tokens)
    text = tokenizer.detokenize(output)[0]  # shape: ()

    tokens = tokenizer.lookup(output)[0]

    _, attention_weights = self.transformer([tokenizer.tokenize(["I am fine"]).to_tensor(), encoder_input, output[:,:-1]], training=False)

    return text, tokens, attention_weights

In [ ]:
translator = Translator(tokenizer, transformer)

def print_translation(sentence, tokens,):
  print(f'{"Input:":15s}: {sentence}')
  print(f'{"Prediction":15s}: {tokens.numpy().decode("utf-8")}')

In [ ]:
sentence = 'can I go park mom'
translated_text, _, _  = translator(tf.constant(sentence))
print_translation(sentence, translated_text)

Input:         : can I go park mom
Prediction     : can i go to the park of mom


# Saving the model

In [ ]:
transformer.save_weights("/drive/MyDrive/twct2weights")
transformer.save("/drive/MyDrive/txct2")